In [ ]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()            #size 3x56x56 input
    self.conv1 = nn.Conv2d(3,4,3)        #four 54x54 feature maps
    self.pool = nn.MaxPool2d(2,2)        #four 27x27 feature maps
    self.conv2 = nn.Conv2d(4,8,5)        #eight 23x23 feature maps
    self.fc1 = nn.Linear(8*11*11, 100)
    self.fc2 = nn.Linear(100, 10)
  def forward(self, x):
    x = self.pool(activation(self.conv1(x)))
    x = self.pool(activation(self.conv2(x)))
    x = x.view(-1, 8*11*11)
    x = activation(self.fc1(x))
    x = self.fc2(x)
    return x